In [ ]:
%load_ext autoreload
%autoreload 2

# chat

> A simple interface for the Mistral API.

> This notebook follows the [fastai style guide](https://docs.fast.ai/dev/style.html). (perhaps move this to the index page only.)

In [ ]:
#| default_exp chat 

In [ ]:
#| export
def foo(): pass

## Export -

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()